*Import all the things that we need*

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: \ 
The environment is inconsistent, please check the package plan carefully
The following packages are causing the inconsistency:

  - defaults/noarch::ibm-wsrt-py37main-keep==0.0.0=2155
  - defaults/noarch::ibm-wsrt-py37main-main==custom=2155
  - conda-forge/linux-64::pytorch==1.8.0=cpu_py37ha70c682_1
done

# All requested packages already installed.

Libraries imported.


*Upload the file containing Canada's Postal Codes Information*

In [2]:
# The code was removed by Watson Studio for sharing.

In [3]:
#Get the neighbourhood coordiantes#
df_coordinates_raw = pd.read_csv(streaming_body_2, sep="\t", header=None)
df_coordinates_raw.head()

,0,1,2,3,4,5,6,7,8,9,10,11
0,CA,T0A,Eastern Alberta (St. Paul),Alberta,AB,NaN,NaN,NaN,NaN,54.7660,-111.7174,6.0
1,CA,T0B,Wainwright Region (Tofield),Alberta,AB,NaN,NaN,NaN,NaN,53.0727,-111.5816,6.0
2,CA,T0C,Central Alberta (Stettler),Alberta,AB,NaN,NaN,NaN,NaN,52.1431,-111.6941,5.0
3,CA,T0E,Western Alberta (Jasper),Alberta,AB,NaN,NaN,NaN,NaN,53.6758,-115.0948,5.0
4,CA,T0G,North Central Alberta (Slave Lake),Alberta,AB,NaN,NaN,NaN,NaN,55.6993,-114.4529,6.0


In [4]:
#Select only the columns that we need#
df_coordinates=df_coordinates_raw[[1,3,2,9,10]]
df_coordinates = df_coordinates.rename(columns={1: 'Postal Code', 3: 'Borough', 2: 'Neighbourhood', 9: 'Latitude', 10: 'Longtitude'})
df_coordinates.tail()

,Postal Code,Borough,Neighbourhood,Latitude,Longtitude
1651,S9V,Saskatchewan,Lloydminster,53.2835,-110.0016
1652,S9X,Saskatchewan,Meadow Lake,54.1335,-108.4347
1653,Y0A,Yukon,Southeastern Yukon (Watson Lake),61.5793,-131.1481
1654,Y0B,Yukon,Central Yukon (Dawson City),64.6450,-137.5360
1655,Y1A,Yukon,Whitehorse,60.7161,-135.0537


In [5]:
#Select the neighborhoods in Vancouver, only#
vancouver_data = df_coordinates[df_coordinates['Neighbourhood'].str.contains('Vancouver')]
vancouver_data.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longtitude
154,V5K,British Columbia,Vancouver (North Hastings-Sunrise),49.2807,-123.0397
155,V5L,British Columbia,Vancouver (North Grandview-Woodlands),49.2795,-123.0667
156,V5M,British Columbia,Vancouver (South Hastings-Sunrise / North Renf...,49.2600,-123.0398
157,V5N,British Columbia,Vancouver (South Grandview-Woodlands / NE Kens...,49.2551,-123.0667
158,V5P,British Columbia,Vancouver (SE Kensington / Victoria-Fraserview),49.2220,-123.0683


In [6]:
vancouver_data.shape

(44, 5)

In [7]:
print("There are " + str(len(vancouver_data)) + " neighborhoods in Vancouver")

There are 44 neighborhoods in Vancouver


*Insert the Cordiantes of Vancouver*

In [8]:
address = 'Vancouver, CA'

latitude = 49.282729
longitude = -123.120738
print('The geograpical coordinate of Vancouver are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Vancouver are 49.282729, -123.120738.


In [9]:
# create map of Vancouver showing the different neighborhoods#
map_vancouver = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, label in zip(vancouver_data['Latitude'], vancouver_data['Longtitude'], vancouver_data['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_vancouver)  
    
map_vancouver

In [10]:
# The code was removed by Watson Studio for sharing.

In [11]:
#Find all the Venues in Toronto#
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [12]:
vancouver_venues = getNearbyVenues(names=vancouver_data['Neighbourhood'],
                                   latitudes=vancouver_data['Latitude'],
                                   longitudes=vancouver_data['Longtitude']
                                  )

Vancouver (North Hastings-Sunrise)
Vancouver (North Grandview-Woodlands)
Vancouver (South Hastings-Sunrise / North Renfrew-Collingwood)
Vancouver (South Grandview-Woodlands / NE Kensington)
Vancouver (SE Kensington / Victoria-Fraserview)
Vancouver (South Renfrew-Collingwood)
Vancouver (Killarney)
Vancouver (East Mount Pleasant)
Vancouver (West Kensington / NE Riley Park-Little Mountain)
Vancouver (SE Riley Park-Little Mountain / SW Kensington / NE Oakridge / North Sunset)
Vancouver (SE Oakridge / East Marpole / South Sunset)
Vancouver (West Mount Pleasant / West Riley Park-Little Mountain)
Vancouver (East Fairview / South Cambie)
Vancouver (Strathcona / Chinatown / Downtown Eastside)
Vancouver (NE Downtown / Harbour Centre / Gastown / Yaletown)
Vancouver (Waterfront / Coal Harbour / Canada Place)
Vancouver (South West End)
Vancouver (North West End / Stanley Park)
Vancouver (West Fairview / Granville Island / NE Shaughnessy)
Vancouver (NW Shaughnessy / East Kitsilano / Quilchena)
Vanco

In [13]:
vancouver_venues.shape

(559, 7)

*Foursquare lists 798 venues in Vancouver. Let us have a look at a few of them*

In [14]:
vancouver_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Vancouver (North Hastings-Sunrise),49.2807,-123.0397,The Fair at the PNE,49.282971,-123.042109,Fair
1,Vancouver (North Hastings-Sunrise),49.2807,-123.0397,Wooden Roller Coaster,49.281744,-123.035128,Theme Park Ride / Attraction
2,Vancouver (North Hastings-Sunrise),49.2807,-123.0397,Livestock Barns,49.284037,-123.039278,Farm
3,Vancouver (North Hastings-Sunrise),49.2807,-123.0397,Playland,49.281924,-123.036258,Theme Park
4,Vancouver (North Hastings-Sunrise),49.2807,-123.0397,Italia Bakery,49.280940,-123.045885,Bakery


In [15]:
#group venues by neighbourhood#
vancouver_grouped_i = vancouver_venues.groupby('Neighborhood').count()
vancouver_grouped_i

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
North Vancouver East Central,1,1,1,1,1,1
North Vancouver North Central,3,3,3,3,3,3
North Vancouver Northwest,19,19,19,19,19,19
North Vancouver Northwest Central,3,3,3,3,3,3
North Vancouver South Central,4,4,4,4,4,4
North Vancouver Southwest,12,12,12,12,12,12
North Vancouver Southwest Central,1,1,1,1,1,1
Vancouver (Bentall Centre),9,9,9,9,9,9
Vancouver (Central Kitsilano),28,28,28,28,28,28


In [16]:
vancouver_grouped_i.shape

(40, 6)

*From Foursqaure we found venues from 40 neighborhoods<br>
Our Postal Codes Files has 44 neighborhoods<br>
This means that there are no venues on Foursquare for 4 neighborhoods*

*Let's look at all the different venue categories*

In [17]:
vancouver_venues['Venue Category'].unique()

array(['Fair', 'Theme Park Ride / Attraction', 'Farm', 'Theme Park',
       'Bakery', 'Beer Garden', 'Soccer Field', 'Park', 'Amphitheater',
       'Gas Station', 'Event Space', 'Sandwich Place',
       'Vietnamese Restaurant', 'Sushi Restaurant', 'Farmers Market',
       'Stadium', 'Bus Line', 'Inn', 'Pizza Place',
       'Portuguese Restaurant', 'Deli / Bodega', 'Coffee Shop',
       'Italian Restaurant', 'Theater', 'Fried Chicken Joint', 'Brewery',
       'Thai Restaurant', 'Café', 'Breakfast Spot', 'Japanese Restaurant',
       'Athletics & Sports', 'Thrift / Vintage Store',
       'Chinese Restaurant', 'Indian Restaurant', 'Asian Restaurant',
       'Electronics Store', 'Fish & Chips Shop', 'Convenience Store',
       'Pet Store', 'Liquor Store', 'Office', 'Grocery Store',
       'Paper / Office Supplies Store', 'Mexican Restaurant',
       'Insurance Office', 'Furniture / Home Store', 'Mobile Phone Shop',
       'Bus Stop', 'Metro Station', 'Storage Facility', 'Restaurant',
     

*Clean up the neighborhood name to remove the leading "Vancouver" and the parentheses*

In [18]:
vancouver_venues['Col1'] = vancouver_venues['Neighborhood'].str.strip('Vancouver')
vancouver_venues['Col3'] = vancouver_venues['Col1'].str.replace('(','')
vancouver_venues['Col4'] = vancouver_venues['Col3'].str.replace(')','')
vancouver_venues.head()

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/ipykernel/__main__.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  from ipykernel import kernelapp as app
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/ipykernel/__main__.py:3: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  app.launch_new_instance()


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,Col1,Col3,Col4
0,Vancouver (North Hastings-Sunrise),49.2807,-123.0397,The Fair at the PNE,49.282971,-123.042109,Fair,(North Hastings-Sunrise),North Hastings-Sunrise),North Hastings-Sunrise
1,Vancouver (North Hastings-Sunrise),49.2807,-123.0397,Wooden Roller Coaster,49.281744,-123.035128,Theme Park Ride / Attraction,(North Hastings-Sunrise),North Hastings-Sunrise),North Hastings-Sunrise
2,Vancouver (North Hastings-Sunrise),49.2807,-123.0397,Livestock Barns,49.284037,-123.039278,Farm,(North Hastings-Sunrise),North Hastings-Sunrise),North Hastings-Sunrise
3,Vancouver (North Hastings-Sunrise),49.2807,-123.0397,Playland,49.281924,-123.036258,Theme Park,(North Hastings-Sunrise),North Hastings-Sunrise),North Hastings-Sunrise
4,Vancouver (North Hastings-Sunrise),49.2807,-123.0397,Italia Bakery,49.280940,-123.045885,Bakery,(North Hastings-Sunrise),North Hastings-Sunrise),North Hastings-Sunrise


In [19]:
vancouver_venues_2 = vancouver_venues.drop(['Col1','Col3'], axis=1)
vancouver_venues_2.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,Col4
0,Vancouver (North Hastings-Sunrise),49.2807,-123.0397,The Fair at the PNE,49.282971,-123.042109,Fair,North Hastings-Sunrise
1,Vancouver (North Hastings-Sunrise),49.2807,-123.0397,Wooden Roller Coaster,49.281744,-123.035128,Theme Park Ride / Attraction,North Hastings-Sunrise
2,Vancouver (North Hastings-Sunrise),49.2807,-123.0397,Livestock Barns,49.284037,-123.039278,Farm,North Hastings-Sunrise
3,Vancouver (North Hastings-Sunrise),49.2807,-123.0397,Playland,49.281924,-123.036258,Theme Park,North Hastings-Sunrise
4,Vancouver (North Hastings-Sunrise),49.2807,-123.0397,Italia Bakery,49.280940,-123.045885,Bakery,North Hastings-Sunrise


*Merge "vancouver_venues_2" with "vancouver_data" to get the Postal Codes*

In [20]:
final_merged = pd.merge(vancouver_venues_2, vancouver_data, left_on='Neighborhood', right_on='Neighbourhood', how='left')
final_merged.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,Col4,Postal Code,Borough,Neighbourhood,Latitude,Longtitude
0,Vancouver (North Hastings-Sunrise),49.2807,-123.0397,The Fair at the PNE,49.282971,-123.042109,Fair,North Hastings-Sunrise,V5K,British Columbia,Vancouver (North Hastings-Sunrise),49.2807,-123.0397
1,Vancouver (North Hastings-Sunrise),49.2807,-123.0397,Wooden Roller Coaster,49.281744,-123.035128,Theme Park Ride / Attraction,North Hastings-Sunrise,V5K,British Columbia,Vancouver (North Hastings-Sunrise),49.2807,-123.0397
2,Vancouver (North Hastings-Sunrise),49.2807,-123.0397,Livestock Barns,49.284037,-123.039278,Farm,North Hastings-Sunrise,V5K,British Columbia,Vancouver (North Hastings-Sunrise),49.2807,-123.0397
3,Vancouver (North Hastings-Sunrise),49.2807,-123.0397,Playland,49.281924,-123.036258,Theme Park,North Hastings-Sunrise,V5K,British Columbia,Vancouver (North Hastings-Sunrise),49.2807,-123.0397
4,Vancouver (North Hastings-Sunrise),49.2807,-123.0397,Italia Bakery,49.280940,-123.045885,Bakery,North Hastings-Sunrise,V5K,British Columbia,Vancouver (North Hastings-Sunrise),49.2807,-123.0397


*Drop unecessary columns and rename necessary onew to get a tidy dataframe*

In [21]:
final_merged_2 = final_merged.drop(['Neighbourhood', 'Borough', 'Latitude', 'Longtitude'], axis=1)
final_merged_2.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,Col4,Postal Code
0,Vancouver (North Hastings-Sunrise),49.2807,-123.0397,The Fair at the PNE,49.282971,-123.042109,Fair,North Hastings-Sunrise,V5K
1,Vancouver (North Hastings-Sunrise),49.2807,-123.0397,Wooden Roller Coaster,49.281744,-123.035128,Theme Park Ride / Attraction,North Hastings-Sunrise,V5K
2,Vancouver (North Hastings-Sunrise),49.2807,-123.0397,Livestock Barns,49.284037,-123.039278,Farm,North Hastings-Sunrise,V5K
3,Vancouver (North Hastings-Sunrise),49.2807,-123.0397,Playland,49.281924,-123.036258,Theme Park,North Hastings-Sunrise,V5K
4,Vancouver (North Hastings-Sunrise),49.2807,-123.0397,Italia Bakery,49.280940,-123.045885,Bakery,North Hastings-Sunrise,V5K


In [22]:
final_merged_3 = final_merged_2.drop(['Neighborhood'], axis=1)
final_merged_3= final_merged_3.rename(columns = {'Col4':'Neighborhood'})
final_merged_3.head()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,Neighborhood,Postal Code
0,49.2807,-123.0397,The Fair at the PNE,49.282971,-123.042109,Fair,North Hastings-Sunrise,V5K
1,49.2807,-123.0397,Wooden Roller Coaster,49.281744,-123.035128,Theme Park Ride / Attraction,North Hastings-Sunrise,V5K
2,49.2807,-123.0397,Livestock Barns,49.284037,-123.039278,Farm,North Hastings-Sunrise,V5K
3,49.2807,-123.0397,Playland,49.281924,-123.036258,Theme Park,North Hastings-Sunrise,V5K
4,49.2807,-123.0397,Italia Bakery,49.280940,-123.045885,Bakery,North Hastings-Sunrise,V5K


In [23]:
final_reordered = final_merged_3[['Postal Code','Neighborhood','Neighborhood Latitude','Neighborhood Longitude','Venue', 'Venue Latitude', 'Venue Longitude', 'Venue Category']]
final_reordered.head()

,Postal Code,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,V5K,North Hastings-Sunrise,49.2807,-123.0397,The Fair at the PNE,49.282971,-123.042109,Fair
1,V5K,North Hastings-Sunrise,49.2807,-123.0397,Wooden Roller Coaster,49.281744,-123.035128,Theme Park Ride / Attraction
2,V5K,North Hastings-Sunrise,49.2807,-123.0397,Livestock Barns,49.284037,-123.039278,Farm
3,V5K,North Hastings-Sunrise,49.2807,-123.0397,Playland,49.281924,-123.036258,Theme Park
4,V5K,North Hastings-Sunrise,49.2807,-123.0397,Italia Bakery,49.280940,-123.045885,Bakery


*Selecting only the data that have to do with Restaurants<br>   
Using only the "Venue Category" names that contain the word "Restaurant"<br>
leaves out Burger (joint), Fried Chicken (joint), Pizza (place) and diner<br>
which are really popular in business sales. So, let us add them as well.*   

In [24]:
vancouver_venues_new=final_reordered[final_reordered['Venue Category'].str.contains('Restaurant|Pizza|Burger|Fried Chicken|Diner')]
vancouver_venues_new.head()

,Postal Code,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
12,V5K,North Hastings-Sunrise,49.2807,-123.0397,Pho Don,49.280947,-123.045543,Vietnamese Restaurant
13,V5K,North Hastings-Sunrise,49.2807,-123.0397,Blowfish Sushi,49.281011,-123.045696,Sushi Restaurant
22,V5K,North Hastings-Sunrise,49.2807,-123.0397,Dijo's Pizza,49.277315,-123.038115,Pizza Place
23,V5K,North Hastings-Sunrise,49.2807,-123.0397,Senhor Rooster,49.277320,-123.038085,Portuguese Restaurant
26,V5L,North Grandview-Woodlands,49.2795,-123.0667,Pepino’s,49.279134,-123.070567,Italian Restaurant


In [25]:
#Group the data by Neighborhood#
vancouver_venues_new['No of Restaurants']=vancouver_venues_new.groupby(["Neighborhood"])['Venue'].transform('count')
vancouver_venues_new.head()

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app


,Postal Code,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,No of Restaurants
12,V5K,North Hastings-Sunrise,49.2807,-123.0397,Pho Don,49.280947,-123.045543,Vietnamese Restaurant,4
13,V5K,North Hastings-Sunrise,49.2807,-123.0397,Blowfish Sushi,49.281011,-123.045696,Sushi Restaurant,4
22,V5K,North Hastings-Sunrise,49.2807,-123.0397,Dijo's Pizza,49.277315,-123.038115,Pizza Place,4
23,V5K,North Hastings-Sunrise,49.2807,-123.0397,Senhor Rooster,49.277320,-123.038085,Portuguese Restaurant,4
26,V5L,North Grandview-Woodlands,49.2795,-123.0667,Pepino’s,49.279134,-123.070567,Italian Restaurant,11


In [26]:
#Create dataframe with single row per neighbourhood#
new_restaurants = vancouver_venues_new.drop_duplicates('Neighborhood')
new_restaurants.head()

,Postal Code,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,No of Restaurants
12,V5K,North Hastings-Sunrise,49.2807,-123.0397,Pho Don,49.280947,-123.045543,Vietnamese Restaurant,4
26,V5L,North Grandview-Woodlands,49.2795,-123.0667,Pepino’s,49.279134,-123.070567,Italian Restaurant,11
59,V5M,South Hastings-Sunrise / North Renfrew-Collin...,49.2600,-123.0398,Vallarta's Mexican Restaurant,49.262232,-123.044736,Mexican Restaurant,6
95,V5P,SE Kensington / Victoria-Fraserview,49.2220,-123.0683,Tibisti Foods & Grill,49.220397,-123.065821,Middle Eastern Restaurant,9
114,V5R,South Renfrew-Collingwood,49.2397,-123.0407,Chili Pepper House,49.236270,-123.041805,Asian Restaurant,1


In [27]:
#Keep only the columns that we need and print the entire dataframe#
final_data = new_restaurants[['Postal Code', 'Neighborhood','Neighborhood Longitude', 'Neighborhood Latitude', 'No of Restaurants']]
final_data

,Postal Code,Neighborhood,Neighborhood Longitude,Neighborhood Latitude,No of Restaurants
12,V5K,North Hastings-Sunrise,-123.0397,49.2807,4
26,V5L,North Grandview-Woodlands,-123.0667,49.2795,11
59,V5M,South Hastings-Sunrise / North Renfrew-Collin...,-123.0398,49.2600,6
95,V5P,SE Kensington / Victoria-Fraserview,-123.0683,49.2220,9
114,V5R,South Renfrew-Collingwood,-123.0407,49.2397,1
120,V5S,Killarney,-123.0380,49.2175,4
139,V5T,East Mount Pleasant,-123.0923,49.2620,8
164,V5V,West Kensington / NE Riley Park-Little Mountain,-123.0913,49.2480,8
184,V5W,SE Riley Park-Little Mountain / SW Kensington...,-123.0917,49.2327,12
203,V5X,SE Oakridge / East Marpole / South Sunset,-123.0979,49.2156,2


*And the top 5 areas with the most Restaurants are:*

In [28]:
final_data_sorted = final_data.sort_values(by=['No of Restaurants'], ascending=False)
final_data_sorted.head()

,Postal Code,Neighborhood,Neighborhood Longitude,Neighborhood Latitude,No of Restaurants
312,V6E,South West End,-123.1298,49.2833,15
445,V6Z,SW Downtown,-123.1300,49.2767,14
184,V5W,SE Riley Park-Little Mountain / SW Kensington...,-123.0917,49.2327,12
26,V5L,North Grandview-Woodlands,-123.0667,49.2795,11
421,V6P,SE Kerrisdale / SW Oakridge / West Marpole,-123.1396,49.2151,11


*Split the Neighborhoods into 4 quartiles, based on the Number of Restaurants*

In [29]:
final_data['No of Restaurants Quartiles'] = pd.qcut(final_data['No of Restaurants'], 4, labels = False)
final_data.head()

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


,Postal Code,Neighborhood,Neighborhood Longitude,Neighborhood Latitude,No of Restaurants,No of Restaurants Quartiles
12,V5K,North Hastings-Sunrise,-123.0397,49.2807,4,1
26,V5L,North Grandview-Woodlands,-123.0667,49.2795,11,3
59,V5M,South Hastings-Sunrise / North Renfrew-Collin...,-123.0398,49.2600,6,2
95,V5P,SE Kensington / Victoria-Fraserview,-123.0683,49.2220,9,3
114,V5R,South Renfrew-Collingwood,-123.0407,49.2397,1,0


***Create a map of Vancouver***

In [30]:
map_vancouver_2 = folium.Map(location=[latitude, longitude], zoom_start=11)
map_vancouver_2

***Colour code the 4 Groups and add the neighborhoods on the map
a lager label indicates a bigger number of restaurants***

In [31]:
colordict = {0: 'blue', 1: 'darkgreen', 2: 'orange', 3: 'red'}

In [32]:
for lat, lon, Quartiles, Number, Location in zip(final_data['Neighborhood Latitude'], final_data['Neighborhood Longitude'], final_data['No of Restaurants Quartiles'], final_data['No of Restaurants'], final_data['Neighborhood']):
        folium.CircleMarker(
        [lat, lon],
        radius=5+3*Quartiles,
        popup = (str(Location) + '<br>'
                 'Number of Restaurants: ' + str(Number)),
        color='b',
        key_on = Quartiles,
        threshold_scale=[0,1,2,3],
        fill_color=colordict[Quartiles],
        fill=True,
        fill_opacity=0.7
        ).add_to(map_vancouver_2)
map_vancouver_2

***Store No of Restaurants per Neighborhood in Datframe for Presentation***

In [33]:
final_data_no_coordinates = final_data[['Postal Code', 'Neighborhood', 'No of Restaurants']]
final_data_no_coordinates_sorted = final_data_no_coordinates.sort_values(by = 'No of Restaurants', ascending = False)
final_data_no_coordinates_sorted

,Postal Code,Neighborhood,No of Restaurants
312,V6E,South West End,15
445,V6Z,SW Downtown,14
184,V5W,SE Riley Park-Little Mountain / SW Kensington...,12
26,V5L,North Grandview-Woodlands,11
421,V6P,SE Kerrisdale / SW Oakridge / West Marpole,11
95,V5P,SE Kensington / Victoria-Fraserview,9
379,V6K,Central Kitsilano,9
139,V5T,East Mount Pleasant,8
164,V5V,West Kensington / NE Riley Park-Little Mountain,8
253,V6B,NE Downtown / Harbour Centre / Gastown / Yale...,8


*Explore what type of Reastaurant is more common in Vancouver<br>
and some useful stastics for the final report*

In [34]:
howmany = vancouver_venues_new['Venue Category'].value_counts().to_frame()
howmany.head(5)

,Venue Category
Chinese Restaurant,18
Restaurant,15
Pizza Place,14
Sushi Restaurant,13
Japanese Restaurant,12


In [35]:
howmany2 = howmany.iloc[5:10]
howmany2

,Venue Category
Vietnamese Restaurant,12
Indian Restaurant,8
Asian Restaurant,6
Fast Food Restaurant,5
Burger Joint,5


In [36]:
howmany3 = howmany.iloc[10:15]
howmany3

,Venue Category
Diner,5
Greek Restaurant,5
Italian Restaurant,4
Mexican Restaurant,4
American Restaurant,3


In [37]:
howmany4 = howmany.iloc[15:20]
howmany4

,Venue Category
French Restaurant,3
Seafood Restaurant,3
Fried Chicken Joint,3
Ramen Restaurant,3
Middle Eastern Restaurant,3


In [38]:
howmany5 = howmany.iloc[20:26]
howmany5

,Venue Category
Thai Restaurant,3
Lebanese Restaurant,2
Filipino Restaurant,2
Ethiopian Restaurant,2
Portuguese Restaurant,2
Vegetarian / Vegan Restaurant,2


In [39]:
howmany.shape

(36, 1)

In [40]:
vancouver_venues_new.shape

(167, 9)